## 构造 FaissDB

### 从 Embeddings 构建文档

In [1]:
from illufly.embeddings import TextEmbeddings
from illufly.rag import MarkMeta, FaissDB

# 导入文档
local = MarkMeta(chunk_size=550)
local(verbose=True)

[INFO] 已成功加载文件 __DOCS__/./001.md ，其中包含 10 个片段。
[INFO] 已成功加载文件 __DOCS__/./设计理念.md ，其中包含 14 个片段。
[INFO] 已成功加载文件 __DOCS__/./推理模式.md ，其中包含 19 个片段。


[Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source']),
 Document(text="# illufly 设计理念介绍...", meta=['source', 'id']),
 Document(text="一般的开发框架们为了开发者使用，会提供自己的类定义，例如用下面的代码来替代：...", meta=['source', 'id']),
 Document(text="现在，要求你准确记住这些类的名字以及该从哪里引用，就开始慢慢形成挑战了。...", meta=['source', 'id']),
 Document(text="'role': 'user',...", meta=['source', 'id']),
 Document(text="# 常见的智能体推理模式和 illufly 的实现...", meta

In [2]:
# 文档嵌入
emb = TextEmbeddings()
emb(local.documents, verbose=True)

[Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings']),
 Document(text="# illufly 设计理念介绍...", meta=['source', 'id', 'embeddings']),
 Document(text="一般的开发框架们为了开发者使用，会提供自己的类定义，例如用下面的代码来替代：...", meta=['source', 'id', 'embeddings']),
 Document(text="现在，要求你准确记住

In [3]:
# 文档入库
faiss = FaissDB(emb, train=False)

# 查询
faiss("怎么做披萨？", verbose=True)

[INFO] 查询到5条结果


[Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings', 'distance']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings', 'distance']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings', 'distance']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings', 'distance']),
 Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings', 'distance'])]

### 从 VectorDB 添加文本

In [5]:
from illufly.embeddings import TextEmbeddings
from illufly.rag import FaissDB

# 文档嵌入
db = FaissDB(TextEmbeddings())
db.load_text("普鸿是一家做数字消防业务的公司")
db.load_text("幻蝶是一家AI技术公司")

# 查询
db("普鸿是啥？", top_k=5, verbose=True)

[INFO] 查询到2条结果


[Document(text="普鸿是一家做数字消防业务的公司", meta=['raw_meta', 'id', 'source', 'embeddings', 'distance']),
 Document(text="幻蝶是一家AI技术公司", meta=['raw_meta', 'id', 'source', 'embeddings', 'distance'])]

### 从 VectorDB 添加本地文件

In [6]:
from illufly.embeddings import TextEmbeddings
from illufly.rag import FaissDB

# 文档嵌入
db = FaissDB(TextEmbeddings())
db.load("./", chunk_size=550)
# 查询
db("披萨要烤几分钟？", top_k=1)

[Document(text="**Question:**...", meta=['tag', 'id', 'source', 'embeddings', 'distance'])]

## 相似性比较实验

1. 如果文本完全相同，距离就是 (0)
2. 如果包含相同文本，混入没有实体的文本，即使很多，距离仍相近 (0.29)
3. 如果包含相同文本，混入其他实体后的文本，哪怕很少，距离依然增加 (0.43)
4. 如果表达接近，且实体不多，依然距离较近 (0.34)
5. 如果表达接近，但混入文本多，实体也多，距离就非常大 (0.94)

In [2]:
from illufly.rag import FaissDB, TextEmbeddings

# 文档嵌入
db = FaissDB(embeddings=TextEmbeddings())
db.load_text("普鸿是一家做数字消防业务的公司")
db.load_text("普鸿是一家做数字消防业务的公司，在广州做消防业务")
db.load_text("普鸿是一家做数字消防业务的公司，在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点")
db.load_text("普鸿是一家做数字消防业务的公司" + "在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点"*10)
db.load_text("普鸿是一家做数字消防业务的公司, 幻蝶是一家AI技术公司")
db.load_text("普鸿在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点")
db.load_text("普鸿做消防业务, 幻蝶做AI的，鸿蒙做政企服务")
db.load_text("2024年新成立的，主要股东有广州鸿蒙信息科技有限公司，普鸿是一家做数字消防业务的公司，是2024年新成立的，主要股东有广州鸿蒙信息科技有限公司")
db.load_text("幻蝶的AI很厉害"*15 + "消防业务")
db.load_text("幻蝶的AI很厉害"*15)

# 查询
for x in db("普鸿是一家做数字消防业务的公司", top_k=10, verbose=True):
    print(x.meta['distance'], ">>", x.text)

[RUNNABLE] FaissDB.4817467424
[INFO] 查询到10条结果
0.0 >> 普鸿是一家做数字消防业务的公司
0.079405695 >> 普鸿是一家做数字消防业务的公司，在广州做消防业务
0.17805657 >> 普鸿是一家做数字消防业务的公司，在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点
0.29316282 >> 普鸿是一家做数字消防业务的公司在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点
0.33564395 >> 普鸿在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点
0.34367657 >> 普鸿是一家做数字消防业务的公司, 幻蝶是一家AI技术公司
0.374036 >> 2024年新成立的，主要股东有广州鸿蒙信息科技有限公司，普鸿是一家做数字消防业务的公司，是2024年新成立的，主要股东有广州鸿蒙信息科技有限公司
0.43234313 >> 普鸿做消防业务, 幻蝶做AI的，鸿蒙做政企服务
0.94430184 >> 幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害消防业务
1.2295747 >> 幻蝶的AI很厉害幻蝶的AI很厉

In [9]:
from illufly.utils import extract_segments

text = """
# python

## 呵呵
```python
from illufly.chat import ChatQwen
```

# 正文
```markdown
hi,
这是个啥？
```
"""

extract_segments(text, ("```", "```"), mode="single", include_markers=False)

['from illufly.chat import ChatQwen\n```\n\n# 正文\n```markdown\nhi,\n这是个啥？\n```\n']

In [8]:
extract_segments(text, ("```xx", "```"), mode="single", include_markers=True)


['\n# python\n\n## 呵呵\n```python\nfrom illufly.chat import ChatQwen\n```\n\n# 正文\n```markdown\nhi,\n这是个啥？\n```\n']